<a href="https://colab.research.google.com/github/MateusMaruzka/lowcostsensors_tensorflow/blob/main/calib_lowcostsensors_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando as dependências:


1.   Tensorflow Decision Forests



In [8]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, \
                                    validation_curve, cross_val_score, \
                                    cross_validate, \
                                    RepeatedKFold


In [17]:
df = pd.read_csv('sample_data/dataset.csv')
df.set_index('Date/Time', inplace = True)
df.dropna(inplace=True)
df.head()

,temp_iat,o3_iat,co_iat,no_iat,no2_iat,nox_iat,so2_iat,ch4_iat,nmhc_iat,thc_iat,...,temperatura_envcity,umidade_envcity,co_thermo,o3_thermo,no2_thermo,so2_thermo,no_thermo,rh_thermo,pressure_thermo,sensorTemp_thermo
Date/Time,,,,,,,,,,,,,,,,,,,,,
2022-05-02 16:00:00+00:00,26.6,29.25,0.0,2.28,5.32,7.59,1.74,0.09,0.07,0.16,...,26.000000,70.833333,571.500000,12.021500,3.189000,35.806167,0.0,62.895333,92.648000,31.309333
2022-05-25 14:00:00+00:00,24.1,22.13,0.0,2.27,2.13,4.39,0.47,0.09,0.07,0.16,...,24.000000,50.500000,176.000000,25.142333,0.269833,11.167667,0.0,45.622667,93.336667,34.691667
2022-05-25 15:00:00+00:00,24.3,20.82,0.0,2.63,3.36,5.99,0.79,0.09,0.07,0.16,...,24.800000,50.600000,206.833333,26.390167,2.244167,19.484667,0.0,46.027000,93.325333,34.336833
2022-05-25 16:00:00+00:00,24.7,22.91,0.0,2.11,2.19,4.30,0.99,0.09,0.07,0.16,...,27.375000,49.125000,269.500000,25.517167,8.797500,28.839000,0.0,47.843500,93.347500,33.347000
2022-05-25 17:00:00+00:00,24.7,20.42,0.0,1.87,2.93,4.80,1.00,0.09,0.07,0.16,...,29.083333,49.666667,407.666667,20.896667,22.361167,43.097833,0.0,53.039333,93.355500,30.386500


In [38]:
label = 'so2_iat'
label_envcity = 'so2_envcity'
Yco = df[label]
Xco = df.loc[Yco.index][[label,'temperatura_envcity','umidade_envcity', 'so2_envcity']]
print(Xco)

df2 = tfdf.keras.pd_dataframe_to_tf_dataset(Xco, task=tfdf.keras.Task.REGRESSION, label=label)
print(df2)

Yco = np.array(Yco).ravel()
Xco = np.array(Xco)

# print("Tam da amostra:",len(Yco)/24, "dias")
# print("Tam da amostra:",len(Xco)/24, "dias")
# Convert the dataset into a TensorFlow dataset.
X_train, X_test, y_train, y_test = train_test_split(Xco, Yco)

# X_train = tf.constant(X_train, dtype=tf.float32)
# y_train = tf.constant(y_train, dtype=tf.float32)

# X_test = tf.constant(X_test, dtype=tf.float32)
# y_test = tf.constant(y_test, dtype=tf.float32)



                           so2_iat  temperatura_envcity  umidade_envcity  \
Date/Time                                                                  
2022-05-02 16:00:00+00:00     1.74            26.000000        70.833333   
2022-05-25 14:00:00+00:00     0.47            24.000000        50.500000   
2022-05-25 15:00:00+00:00     0.79            24.800000        50.600000   
2022-05-25 16:00:00+00:00     0.99            27.375000        49.125000   
2022-05-25 17:00:00+00:00     1.00            29.083333        49.666667   
...                            ...                  ...              ...   
2022-07-28 18:00:00+00:00     2.03            32.062500        63.875000   
2022-07-28 19:00:00+00:00     2.05            31.566038        67.924528   
2022-07-28 20:00:00+00:00     2.11            29.900000        69.800000   
2022-07-28 21:00:00+00:00     2.22            25.760870        69.326087   
2022-07-28 22:00:00+00:00     2.14            25.000000        69.000000   

           

In [46]:
# Train a Random Forest model.
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION, verbose=2)
# model.fit(x=X_train, y=y_train)
model.fit(df2)

Use 2 thread(s) for training
Use /tmp/tmpjbxdghtb as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'temperatura_envcity': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'umidade_envcity': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'so2_envcity': <tf.Tensor 'data:0' shape=(None,) dtype=float64>}
Label: Tensor("data_3:0", shape=(None,), dtype=float64)
Weights: None
Normalized tensor features:
 {'temperatura_envcity': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'umidade_envcity': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_1:0' shape=(None,) dtype=float32>), 'so2_envcity': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_2:0' shape=(None,) dtype=float32>)}
Training dataset read in 0:00:00.246523. Found 346 examples.
Training model...
Standard output detected as not visible to the user e.g. running in a notebook. C

[INFO 2022-11-26T17:48:18.392490664+00:00 kernel.cc:814] Start Yggdrasil model training
[INFO 2022-11-26T17:48:18.392828013+00:00 kernel.cc:815] Collect training examples
[INFO 2022-11-26T17:48:18.392896189+00:00 kernel.cc:423] Number of batches: 1
[INFO 2022-11-26T17:48:18.392903896+00:00 kernel.cc:424] Number of examples: 346
[INFO 2022-11-26T17:48:18.392954205+00:00 kernel.cc:837] Training dataset:
Number of records: 346
Number of columns: 4

Number of columns by type:
	NUMERICAL: 4 (100%)

Columns:

NUMERICAL: 4 (100%)
	0: "so2_envcity" NUMERICAL mean:-2029.3 min:-2468.34 max:8207.22 sd:1303.21
	1: "temperatura_envcity" NUMERICAL mean:19.7478 min:11 max:32.0625 sd:4.48922
	2: "umidade_envcity" NUMERICAL mean:86.3711 min:49.125 max:95 sd:12.7304
	3: "__LABEL" NUMERICAL mean:1.18514 min:-0.35 max:4.54 sd:0.745805

Terminology:
	nas: Number of non-available (i.e. missing) values.
	ood: Out of dictionary.
	manually-defined: Attribute which type is manually defined by the user i.e. the 

Model trained in 0:00:00.476549
Compiling model...
Model compiled.


In [47]:
# Summary of the model structure.
model.summary()

Model: "random_forest_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: REGRESSION
Label: "__LABEL"

Input Features (3):
	so2_envcity
	temperatura_envcity
	umidade_envcity

No weights

Variable Importance: MEAN_MIN_DEPTH:
    1.             "__LABEL"  7.811658 ################
    2.         "so2_envcity"  2.471945 ###
    3.     "umidade_envcity"  1.729726 #
    4. "temperatura_envcity"  1.050651 

Variable Importance: NUM_AS_ROOT:
    1. "temperatura_envcity" 149.000000 ################
    2.     "umidade_envcity" 110.000000 ##########
    3.         "so2_envcity" 41.000000 

Variable Importance: NUM_NODES:
    1. "temperatura_envcity" 6405.000000 ################
    2.         "so2_envcity" 5254.000000 #########
    3.     "umidade_envcity"

In [42]:
# Evaluate the model.
model.evaluate(x=X_test, y=y_test)

# Export the model to a SavedModel.
# model.save("project/model")

ValueError: ignored

In [45]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)